# IB-ActividadLightbeam

En esta libreta vamos a:
1. Cargar y analizar el archivo `ThunderbeamData.json` (o “LightbeamData.json”).
2. Responder a las preguntas:
   - Número total de terceros conectados.
   - Página visitada con más terceros.
   - Dos páginas con más terceros en común.
   - Tercero que aparece más a menudo.


## 1. Preparación del entorno

Importamos las librerías necesarias para el análisis.

In [ ]:
import json
import pandas as pd
from collections import defaultdict, Counter

## 2. Carga de datos

Asumiendo que el archivo JSON tiene una estructura aproximada a:
```json
{
  "sites": [
    {
      "page": "https://www.ejemplo1.com",
      "third_parties": ["ads.ejemplo.com", "analytics.ejemplo2.net", ...]
    },
    {
      "page": "https://www.ejemplo2.com",
      "third_parties": [...]
    },
    ...
  ]
}
```

Si tu archivo tiene otra forma, adapta el código en consecuencia.

In [ ]:
# Lee el archivo JSON (ajusta la ruta a tu caso)
with open('ThunderbeamData.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Supongamos que la clave principal es 'sites'
sites_data = data.get("sites", [])

print(f"Se han cargado {len(sites_data)} entradas de sitios.")

## 3. Análisis de los datos

### 3.1 Número total de terceros distintos
Podemos crear un conjunto global de todos los terceros y contar su cardinalidad.

In [ ]:
all_third_parties = set()

for site in sites_data:
    # Ajustar el campo en función de la clave real del JSON
    tps = site.get('third_parties', [])
    for tp in tps:
        all_third_parties.add(tp)

num_third_parties = len(all_third_parties)
print("Número total de terceros únicos conectados:", num_third_parties)

### 3.2 Página con más terceros
Para cada sitio, contamos cuántos terceros se han detectado, y nos quedamos con el máximo.

In [ ]:
page_to_count = {}

for site in sites_data:
    page = site.get('page', 'Unknown')
    tps = site.get('third_parties', [])
    page_to_count[page] = len(tps)

# Hallamos la página con mayor número de terceros
max_page = None
max_count = -1

for p, c in page_to_count.items():
    if c > max_count:
        max_count = c
        max_page = p

print("Página con más terceros:", max_page)
print("Número de terceros:", max_count)

### 3.3 Las dos páginas con más terceros en común
Calculamos, para cada par de sitios, el número de terceros compartidos.

In [ ]:
# Primero, creamos un dict con los terceros de cada sitio en forma de set
site_to_thirdset = {}
for site in sites_data:
    page = site.get('page', 'Unknown')
    tps = site.get('third_parties', [])
    site_to_thirdset[page] = set(tps)

# Luego comparamos cada par de páginas
import itertools

max_shared = -1
pages_pair = (None, None)
all_pages = list(site_to_thirdset.keys())

for p1, p2 in itertools.combinations(all_pages, 2):
    shared = site_to_thirdset[p1].intersection(site_to_thirdset[p2])
    shared_count = len(shared)
    if shared_count > max_shared:
        max_shared = shared_count
        pages_pair = (p1, p2)

print("Páginas con más terceros en común:", pages_pair)
print("Número de terceros en común:", max_shared)

### 3.4 Tercero que aparece más a menudo
Contamos en cuántos sitios aparece cada dominio de tercero.

In [ ]:
third_counts = Counter()

for site in sites_data:
    tps = set(site.get('third_parties', []))  # conjunto para no contar duplicados en la misma página
    for tp in tps:
        third_counts[tp] += 1

most_common_third = third_counts.most_common(1)
if most_common_third:
    print("Tercero más frecuente:", most_common_third[0][0])
    print("Aparece en", most_common_third[0][1], "páginas.")
else:
    print("No se encontró ningún tercero.")

## 4. Conclusiones
En este punto, ya podemos extraer la información necesaria para contestar a:
- **Número total de terceros conectados**: `num_third_parties`
- **Página con más terceros y cuántos**: `max_page`, `max_count`
- **Dos páginas con más terceros en común y cuántos**: `pages_pair`, `max_shared`
- **Tercero más frecuente**: `most_common_third`

A partir de estos datos podemos completar la actividad solicitada.